# INFO 2950 Project

Robin Ahn, Yama Bazger, Grace Myers, and Kaylah Nicholson (ordered alphabetically by last name)

**This will serve as a preliminary Jupyter drawing board for our project** 

Create sections and cells as needed



## Section 1
Firstly, we will import all the relevant libraries that will help us with our data analysis here.



In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt 
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

We will now import the CSV files that we will be processing.

In [2]:
korea_purpose = pd.read_csv("/Users/yamabazger/Documents/GitHub/info2950-repo/Korea_Visitors_By_Purpose-final.csv")
korea_gender = pd.read_csv("/Users/yamabazger/Documents/GitHub/info2950-repo/Korea_Visitors_By_Gender-final.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yamabazger/Documents/GitHub/info2950-repo/Korea_Visitors_By_Purpose-final.csv'

The original CSV files contained white spaces in some columns with string types. We will strip the white spaces for consistency and to have a more polished sets of data to work with.

In [ ]:
#Stripping whitespaces for columns 'Continents', 'Country', and 'Purpose'
#for korea_purpose
korea_purpose['Continents'] = korea_purpose['Continents'].str.strip()
korea_purpose['Country'] = korea_purpose['Country'].str.strip()
korea_purpose['Purpose'] = korea_purpose['Purpose'].str.strip()

#Stripping whitespaces for columns 'Continents', 'Country', and 'Gender'
#for korea_gender
korea_gender['Continents'] = korea_gender['Continents'].str.strip()
korea_gender['Country'] = korea_gender['Country'].str.strip()
korea_gender['Gender'] = korea_gender['Gender'].str.strip()

We will check the initial shape of korea_purpose.

In [ ]:
korea_purpose.shape

We will do some initially cleaning of the data, such as removing rows that contain terms "etc".... under Country. ROBIN/GRACE, look at the code and complete the description.

In [ ]:
# Define the conditions for each column using index locations
condition_date = (korea_purpose.iloc[:, 0] == 'sum')  
condition_continent = korea_purpose.iloc[:, 1].isin(['Entire continent', 'year', 'Year and Continent', 'etc', 'Korean'])  # Using index 1 for 'Continent'
condition_country_name = korea_purpose.iloc[:, 2].isin(['Entire continent', 'year', 'Year and Continent', 'etc', 'Korean'])  # Using index 2 for 'Country name'
condition_purpose = korea_purpose.iloc[:, 3].isin(['entire'])

#Rows marked for deletion
rows_to_remove = korea_purpose[condition_date | condition_continent | condition_country_name | condition_purpose].index

#Now the actual deletion
korea_purpose = korea_purpose.drop(rows_to_remove)

Now that we have deleted some. unknown number of rows, we will check the shape of korea_purpose to ensure the number of rows has decreased.

In [ ]:
korea_purpose.shape

In [ ]:
korea_gender.head()

We will repeat the same process with korea_gender.

In [ ]:
korea_gender.shape

In [ ]:
# Define the conditions for each column using index locations
condition_date = (korea_gender.iloc[:, 0] == 'sum')  # Using index 0 for 'Base date'
condition_continent = korea_gender.iloc[:, 1].isin(['Entire continent', 'year', 'Year and Continent', 'etc', 'Korean'])  # Using index 1 for 'Continent'
condition_country_name = korea_gender.iloc[:, 2].isin(['Entire continent', 'year', 'Year and Continent', 'etc', 'Korean'])  # Using index 2 for 'Country name'
condition_purpose = korea_gender.iloc[:, 3].isin(['Stewardess', 'entire'])

#Rows marked for deletion
rows_to_remove = korea_gender[condition_date | condition_continent | condition_country_name | condition_purpose].index

#Now the actual deletion
korea_gender = korea_gender.drop(rows_to_remove)

In [ ]:
korea_gender.shape

In [ ]:
korea_gender.head(-3)

We change the date format for both dataframes to facilitate easier data processing 

In [ ]:
korea_purpose['Date'] = pd.to_datetime(korea_purpose['Date'], format='%Y%m')
korea_gender['Date'] = pd.to_datetime(korea_gender['Date'], format='%Y%m')

In [ ]:
total_visitors_by_gender = korea_gender.groupby('Gender')['Visitors'].sum()
# Create a bar plot
total_visitors_by_gender.plot(kind='bar', color=['blue', 'orange'])

# Add title and labels
plt.title('Total Visitors by Gender')
plt.xlabel('Gender')
plt.ylabel('Total Visitors')

# Show the plot
plt.show()

In [ ]:
total_visitors_date_gender = korea_gender.groupby(['Date', 'Gender'])['Visitors'].sum().unstack()

# Plot the data
total_visitors_date_gender.plot(kind='bar')

# Add title and labels
plt.title('Total Visitors by Gender Across Dates')
plt.xlabel('Date')
plt.ylabel('Total Visitors')
plt.xticks(rotation=45)
# Show the plot
plt.show()

In [ ]:
korea_purpose.head()

In [ ]:
total_visitors_country = korea_purpose.groupby('Country')['Visitors'].sum()

# Select top 5 countries
top_5_countries = total_visitors_country.nlargest(5).index

# Filter the data for the top 5 countries
df_top_5 = korea_purpose[korea_purpose['Country'].isin(top_5_countries)]

# Group the data by 'Country' and 'Date' with six-month intervals and calculate the total visitors for each group
total_visitors_country_time = df_top_5.groupby(['Country', pd.Grouper(key='Date', freq='6M')])['Visitors'].sum().unstack(level=0)

# Plot the data
ax = total_visitors_country_time.plot(kind='line', marker='o', figsize=(12, 6))

# Add title and labels
plt.title('Total Visitors by Country Across Time (Every Six Months)')
plt.xlabel('Date')
plt.ylabel('Total Visitors')

# Show the plot
plt.show()

In [ ]:
# Filter the data for the purpose of 'tourism'
korea_purpose_tourism = korea_purpose[korea_purpose['Purpose'] == 'tourism']

# Compute total visitors for each country with the purpose of tourism
total_visitors_country_tourism = korea_purpose_tourism.groupby('Country')['Visitors'].sum()

# Select top 5 countries
top_5_countries_tourism = total_visitors_country_tourism.nlargest(5).index

# Filter the data for the top 5 countries
korea_purpose_top_5_tourism = korea_purpose_tourism[korea_purpose_tourism['Country'].isin(top_5_countries_tourism)]

# Group the data by 'Country' and 'Date' with six-month intervals and calculate the total visitors for each group
total_visitors_country_time_tourism = korea_purpose_top_5_tourism.groupby(['Country', pd.Grouper(key='Date', freq='3M')])['Visitors'].sum().unstack(level=0)

# Plot the data
ax = total_visitors_country_time_tourism.plot(kind='line', marker='o', figsize=(12, 6))

# Add title and labels
plt.title('Total Visitors for Tourism by Country Across Time (Every Six Months)')
plt.xlabel('Date')
plt.ylabel('Total Visitors for Tourism')

# Show the plot
plt.show()

In [ ]:
merged_df = pd.merge(korea_purpose, korea_gender, on=['Date', 'Continents', 'Country'])

# You can also specify the 'how' parameter to specify the type of merge (inner, outer, left, or right)
# For example:
# merged_df = pd.merge(korea_purpose, korea_gender, on='common_column', how='inner')

# Display the merged DataFrame
merged_df.head()

In [ ]:
# Filter the data for the purpose of 'tourism'
merged_df_tourism = merged_df[merged_df['Purpose'] == 'tourism']

# Compute total visitors for tourism by gender and country
total_visitors_tourism_gender_country = merged_df_tourism.groupby(['Country', 'Gender'])['Visitors_x'].sum().unstack()

# Select top 5 countries based on total visitors for tourism
top_5_countries_tourism = total_visitors_tourism_gender_country.sum(axis=1).nlargest(5).index

# Filter the data for the top 5 countries
total_visitors_tourism_gender_country_top5 = total_visitors_tourism_gender_country.loc[top_5_countries_tourism]

# Plot the data
total_visitors_tourism_gender_country_top5.plot(kind='bar', figsize=(10, 6))
plt.title('Total Visitors for Tourism by Gender for Top 5 Countries')
plt.xlabel('Country')
plt.ylabel('Total Visitors for Tourism')
plt.show()

In [ ]:
# Group by 'Date' and 'Gender', and calculate the sum of 'Visitors'
total_visitors_date_gender = korea_gender.groupby(['Date', 'Gender'])['Visitors'].sum().unstack()

# Resample the data to quarterly frequency
total_visitors_date_gender = total_visitors_date_gender.resample('Q').sum()

# Plot the data
total_visitors_date_gender.plot(kind='bar')

# Add title and labels
plt.title('Total Visitors by Gender Across Dates')
plt.xlabel('Date')
plt.ylabel('Total Visitors')
plt.xticks(rotation=45)
# Show the plot
plt.show()

In [ ]:
#selects the relevant columns of the data, and adds a column to represent the respective country,
#returns the resulting dataframe
def add_column(abbrv, country):
    '''
    Param abbrv: string of the file's abbreviation
    Param country: string of the country's name
    '''
    df = pd.read_csv(abbrv+'_youtube_trending_data.csv') 
    country_df = duckdb.sql("SELECT publishedAt, \
                       trending_date, tags, view_count, likes, \
                       comment_count \
                       FROM df \
                       WHERE CONTAINS(tags, 'k-pop')").df()
    country_df.insert(0, 'country', country) 
    return country_df

In [ ]:
brazil_df = add_column('BR', 'Brazil')
brazil_df

In [ ]:
canada_df = add_column('CA', 'Canada')
canada_df

In [ ]:
denmark_df = add_column('DE', 'Denmark')
denmark_df

In [ ]:
france_df = add_column('FR', 'France')
france_df

In [ ]:
india_df = add_column('IN', 'India')
india_df

In [ ]:
japan_df = add_column('JP', 'Japan')
japan_df

In [ ]:
mexico_df = add_column('MX', 'Mexico')
mexico_df

In [ ]:
russia_df = add_column('RU', 'Russia')
russia_df

In [ ]:
united_states_df = add_column('US', 'US')
united_states_df

In [ ]:
united_kingdom_df = add_column('GB', 'UK')
united_kingdom_df

In [ ]:
combined_df = duckdb.sql("SELECT * FROM brazil_df \
                            UNION ALL \
                            SELECT * FROM canada_df \
                            UNION ALL \
                            SELECT * FROM denmark_df \
                            UNION ALL \
                            SELECT * FROM france_df \
                            UNION ALL \
                            SELECT * FROM united_kingdom_df \
                            UNION ALL \
                            SELECT * FROM india_df \
                            UNION ALL \
                            SELECT * FROM japan_df \
                            UNION ALL \
                            SELECT * FROM mexico_df \
                            UNION ALL \
                            SELECT * FROM russia_df \
                            UNION ALL \
                            SELECT * FROM united_states_df").df()
combined_df

In [ ]:
#creates a pie chart of the data of analyzing a column (category) per country
def create_piechart(df, column):
    '''
    Param df: Pandas dataframe
    Param column: String of the column in df to be analyzed

    '''
    grouped = combined_df.groupby('country')[column].sum()
    plt.pie(grouped,labels=grouped.index,autopct='%.2f%%')
    plt.show()

In [ ]:
create_piechart(combined_df, 'view_count')

In [ ]:
create_piechart(combined_df, 'likes')

In [ ]:
create_piechart(combined_df, 'comment_count')

In [ ]:
result_df = combined_df.groupby('publishedAt')['view_count'].sum().reset_index()
x = result_df['publishedAt']
y = result_df['view_count']
plt.plot(x, y)
plt.xlabel('Time Published')
plt.ylabel('Views')
plt.title('Views 2020-2023')
plt.show()

In [ ]:
result_df = combined_df.groupby('publishedAt')['likes'].sum().reset_index()
x = result_df['publishedAt']
y = result_df['likes']
plt.plot(x, y)
plt.xlabel('Time Published')
plt.ylabel('Likes')
plt.title('Likes 2020-2023')
plt.show()

In [ ]:
result_df = combined_df.groupby('publishedAt')['comment_count'].sum().reset_index()
x = result_df['publishedAt']
y = result_df['comment_count']
plt.plot(x, y)
plt.xlabel('Time Published')
plt.ylabel('Comments')
plt.title('Comments 2020-2023')
plt.show()

In [ ]:
## one with trending date? 